<a href="https://colab.research.google.com/github/Aniket1492/PdfML/blob/main/DemoRecommendationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow

In [7]:
import numpy as np
import tensorflow as tf
# tf.enable_eager_excecution()
print(tf.__version__)


2.15.0


In [11]:
users = ['Ryan','Danielle','Vijay','Chris','Kajal','Navmi','Aniket']
Product = ['Almond Croissant','Black Forest Cookies','Hazelnut Croissant','Marian plum Jam','Strawberry jam (L)','Spinach cake 1 pound 0704','Carrot Cake','Fruitcake Cookies',]
features= ['jam','Croissant','Cookies','Cake']

num_users = len(users)
num_Product = len(Product)
num_features = len(features)
num_recommendations = 2
print(num_users)
print(num_Product)
print(num_features)

7
8
4


In [23]:
user_product = tf.constant([
                  [4,3,0,1,0,0,0,0],
                  [0,5,1,3,0,0,0,2],
                  [0,0,3,5,2,0,0,0],
                  [0,0,0,4,3,2,0,0],
                  [0,4,0,0,2,0,1,5],
                  [0,4,0,2,3,0,5,4],
                  [0,4,0,0,2,1,5,0]],dtype=tf.float32)

In [24]:
Product_feature = tf.constant([
                [0,1,0,0],
                [0,0,1,0],
                [0,1,0,0],
                [1,0,0,0],
                [1,0,0,0],
                [0,0,0,1],
                [0,0,0,1],
                [0,0,1,0]],dtype=tf.float32)

In [25]:
userfeature = tf.matmul(user_product,Product_feature)
userfeature

<tf.Tensor: shape=(7, 4), dtype=float32, numpy=
array([[1., 4., 3., 0.],
       [3., 1., 7., 0.],
       [7., 3., 0., 0.],
       [7., 0., 0., 2.],
       [2., 0., 9., 1.],
       [5., 0., 8., 5.],
       [2., 0., 4., 6.]], dtype=float32)>

In [26]:
userfeature = userfeature/tf.reduce_sum(userfeature,axis =1 ,keepdims = True)
userfeature

<tf.Tensor: shape=(7, 4), dtype=float32, numpy=
array([[0.125     , 0.5       , 0.375     , 0.        ],
       [0.27272728, 0.09090909, 0.6363636 , 0.        ],
       [0.7       , 0.3       , 0.        , 0.        ],
       [0.7777778 , 0.        , 0.        , 0.22222222],
       [0.16666667, 0.        , 0.75      , 0.08333334],
       [0.2777778 , 0.        , 0.44444445, 0.2777778 ],
       [0.16666667, 0.        , 0.33333334, 0.5       ]], dtype=float32)>

In [27]:
top_user_features =tf.nn.top_k(userfeature,num_features)[1]
top_user_features

<tf.Tensor: shape=(7, 4), dtype=int32, numpy=
array([[1, 2, 0, 3],
       [2, 0, 1, 3],
       [0, 1, 2, 3],
       [0, 3, 1, 2],
       [2, 0, 3, 1],
       [2, 0, 3, 1],
       [3, 2, 0, 1]], dtype=int32)>

In [28]:
for i in range(num_users):
  feature_names = [features[int(index)] for index in top_user_features[i]]
  print('{}:{}'.format(users[i],feature_names))

Ryan:['Croissant', 'Cookies', 'jam', 'Cake']
Danielle:['Cookies', 'jam', 'Croissant', 'Cake']
Vijay:['jam', 'Croissant', 'Cookies', 'Cake']
Chris:['jam', 'Cake', 'Croissant', 'Cookies']
Kajal:['Cookies', 'jam', 'Cake', 'Croissant']
Navmi:['Cookies', 'jam', 'Cake', 'Croissant']
Aniket:['Cake', 'Cookies', 'jam', 'Croissant']


In [29]:
users_rating = tf.matmul(userfeature,tf.transpose(Product_feature))
users_rating

<tf.Tensor: shape=(7, 8), dtype=float32, numpy=
array([[0.5       , 0.375     , 0.5       , 0.125     , 0.125     ,
        0.        , 0.        , 0.375     ],
       [0.09090909, 0.6363636 , 0.09090909, 0.27272728, 0.27272728,
        0.        , 0.        , 0.6363636 ],
       [0.3       , 0.        , 0.3       , 0.7       , 0.7       ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.7777778 , 0.7777778 ,
        0.22222222, 0.22222222, 0.        ],
       [0.        , 0.75      , 0.        , 0.16666667, 0.16666667,
        0.08333334, 0.08333334, 0.75      ],
       [0.        , 0.44444445, 0.        , 0.2777778 , 0.2777778 ,
        0.2777778 , 0.2777778 , 0.44444445],
       [0.        , 0.33333334, 0.        , 0.16666667, 0.16666667,
        0.5       , 0.5       , 0.33333334]], dtype=float32)>

In [30]:
user_rating_new = tf.where(tf.equal(user_product,tf.zeros_like(user_product)),
                           users_rating,
                           tf.zeros_like(tf.cast(user_product,tf.float32)))
user_rating_new

<tf.Tensor: shape=(7, 8), dtype=float32, numpy=
array([[0.        , 0.        , 0.5       , 0.        , 0.125     ,
        0.        , 0.        , 0.375     ],
       [0.09090909, 0.        , 0.        , 0.        , 0.27272728,
        0.        , 0.        , 0.        ],
       [0.3       , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.22222222, 0.        ],
       [0.        , 0.        , 0.        , 0.16666667, 0.        ,
        0.08333334, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.2777778 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.16666667, 0.        ,
        0.        , 0.        , 0.33333334]], dtype=float32)>

In [31]:
top_product = tf.nn.top_k(user_rating_new,num_recommendations)[1]
top_product

<tf.Tensor: shape=(7, 2), dtype=int32, numpy=
array([[2, 7],
       [4, 0],
       [0, 1],
       [6, 0],
       [3, 5],
       [5, 0],
       [7, 3]], dtype=int32)>

In [32]:
for i in range(num_users):
  product_names = [Product[int(index)] for index in top_product[i]]
  print('{}:{}'.format(users[i],product_names))

Ryan:['Hazelnut Croissant', 'Fruitcake Cookies']
Danielle:['Strawberry jam (L)', 'Almond Croissant']
Vijay:['Almond Croissant', 'Black Forest Cookies']
Chris:['Carrot Cake', 'Almond Croissant']
Kajal:['Marian plum Jam', 'Spinach cake 1 pound 0704']
Navmi:['Spinach cake 1 pound 0704', 'Almond Croissant']
Aniket:['Fruitcake Cookies', 'Marian plum Jam']
